## LangChain

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

prompt = ChatPromptTemplate.from_template("{topic} 에 대해 쉽게 설명해줘")
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

question = {"topic" : "랭체인"}

response = chain.invoke(question)
print(response)

랭체인(LangChain)은 자연어 처리(NLP) 및 인공지능(AI) 애플리케이션을 개발하기 위한 프레임워크입니다. 주로 대화형 AI, 챗봇, 언어 모델을 구축하는 데 사용됩니다. 이 프레임워크는 다양한 도구와 기술을 통합하여 개발자들이 손쉽게 언어 모델을 활용하고, 이를 기반으로 여러 기능을 구현할 수 있도록 돕습니다.

랭체인의 주요 특징은 다음과 같습니다:

1. **모듈화**: 다양한 기능과 컴포넌트를 모듈화하여 개발자들이 필요에 따라 선택하고 조합할 수 있습니다.
  
2. **데이터 연결**: 외부 데이터 소스와의 연결을 지원하여 언어 모델이 특정 도메인이나 주제에 맞춘 정보에 접근할 수 있게 합니다.

3. **상태 관리**: 대화의 상태를 추적하고 관리할 수 있는 기능을 제공하여 보다 자연스러운 대화 흐름을 구현할 수 있습니다.

4. **사용자 정의**: 개발자가 자신의 요구에 맞게 언어 모델을 커스터마이즈 할 수 있도록 다양한 옵션을 제공합니다.

5. **통합성**: 여러 외부 API나 시스템과의 통합이 용이하여, 실제 애플리케이션에서 활용하기 쉽습니다.

랭체인은 주로 파이썬으로 개발되며, 자연어 이해(NLU), 질문 응답 시스템, 요약 생성 등 다양한 언어 기반 애플리케이션을 만드는 데 유용하게 사용됩니다.


In [10]:
# from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. pdf 문서 로드

loader = PyMuPDFLoader(file_path="data/SPRi AI Brief_10월호_산업동향_1002_F.pdf")
pages = loader.load()
print(f"pdf를 총{len(pages)}개의 페이지로 불러왔습니다.\n")

pdf를 총29개의 페이지로 불러왔습니다.



In [15]:
import tiktoken
from langchain_text_splitters import TokenTextSplitter

encoding = tiktoken.get_encoding('o200k_base') # <Encoding 'o200k_base'>

def num_tokens_from_string(string: str) -> int :
  """ 문자열의 토큰 수를 계산하여 반환"""
  return len(encoding.encode(string))

sample_text = "This splitter is essential for managing LLM context windows and optimizing API costs by counting tokens."

total_tokens = num_tokens_from_string(sample_text)
print(f"원본 텍스트의 총 토큰 수 : {total_tokens}\n")

text_splitter = TokenTextSplitter(
  chunk_size = 15,
  chunk_overlap = 5,
  encoding_name = "o200k_base"
)
  
#2. 텍스트 분할
chunks = text_splitter.split_text(sample_text)

# 3. 결과확인

print(f"TokenTextSplitter 결과 (총 {len(chunks)}개)")
for i, chunk in enumerate(chunks) :
  chunk_tokens = num_tokens_from_string(chunk)
  print(f"[{i+1}] (토큰: {chunk_tokens} {chunk})")


원본 텍스트의 총 토큰 수 : 18

TokenTextSplitter 결과 (총 2개)
[1] (토큰: 15 This splitter is essential for managing LLM context windows and optimizing API costs by)
[2] (토큰: 8  and optimizing API costs by counting tokens.)


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from dotenv import load_dotenv

# pdf 파일 로드
load_dotenv()

file_path = "data/SPRi AI Brief_10월호_산업동향_1002_F.pdf"
loader = PyPDFLoader(file_path)
documents = loader.load()

print(f"---- PDF 문서 로드 완료 ---")
print(f"총 {len(documents)}개의 페이지가 로드 되었습니다.\n")

# 임베딩 모델 초기화

embedding_model = OpenAIEmbeddings(model= "text-embedding-3-small")

# 문서 내용만 추출
documents_texts = [doc.page_content for doc in documents]

document_embeddings = embedding_model.embed_documents(documents_texts)

print("----- 여러 문서 임베딩( embed_documents) 결과 ---")
print(f"총 {len(document_embeddings)}개의 페이지가 임베딩 되었습니다.")
print(f"첫 번째 페이지의 벡터 차원(크기): {len(document_embeddings[0])}")
print(f"첫 번째 페이지의 벡터 차원(앞 5개 값): {document_embeddings[0][:5]}\n")

# 첫번째 페이지 내용 미리보기
print("----- 여러 문서 임베딩( embed_documents) 결과 ---")
print("document_texts[0][:500]")
print("...\n")

# 단일 텍스트 질의 임베딩

query = "AI 산업 동향에 대해 알려주세요"
query_embedding = embedding_model.embed_query(query)

print("----- 단일 질의 임베딩 (embed_query) 결과 ---")
print(f"총 {len(document_embeddings)}개의 페이지가 임베딩 되었습니다.")
print(f"첫 번째 페이지의 벡터 차원(앞 5개 값): {document_embeddings[0][:5]}\n")

In [ ]:
from langchain_openai import HuggingFaceEmbeddings

# pdf 파일 로드
load_dotenv()

file_path = "data/SPRi AI Brief_10월호_산업동향_1002_F.pdf"
loader = PyPDFLoader(file_path)
documents = loader.load()

print(f"---- PDF 문서 로드 완료 ---")
print(f"총 {len(documents)}개의 페이지가 로드 되었습니다.\n")

# 임베딩 모델 초기화

embedding_model = HuggingFaceEmbeddings(
  model_name = 
)

# 문서 내용만 추출
documents_texts = [doc.page_content for doc in documents]

document_embeddings = embedding_model.embed_documents(documents_texts)

print("----- 여러 문서 임베딩( embed_documents) 결과 ---")
print(f"총 {len(document_embeddings)}개의 페이지가 임베딩 되었습니다.")
print(f"첫 번째 페이지의 벡터 차원(크기): {len(document_embeddings[0])}")
print(f"첫 번째 페이지의 벡터 차원(앞 5개 값): {document_embeddings[0][:5]}\n")

# 첫번째 페이지 내용 미리보기
print("----- 여러 문서 임베딩( embed_documents) 결과 ---")
print("document_texts[0][:500]")
print("...\n")

# 단일 텍스트 질의 임베딩

query = "AI 산업 동향에 대해 알려주세요"
query_embedding = embedding_model.embed_query(query)

print("----- 단일 질의 임베딩 (embed_query) 결과 ---")
print(f"총 {len(document_embeddings)}개의 페이지가 임베딩 되었습니다.")
print(f"첫 번째 페이지의 벡터 차원(앞 5개 값): {document_embeddings[0][:5]}\n")

## 벡터스토어

In [ ]:
import psycopg
from dotenv import load_dotenv
import os
from langchain_openai import OpenAIEmbeddings
from langchain_postgres import PGVector
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv()

DB_CONFIG = {
    'host': os.getenv("DB_HOST"),
    'port': 5432,
    'database': os.getenv('DB_NAME'), 
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASS'),
}

COLLECTION_NAME = "vector-db-test"
FILE_PATH = "data/SPRi AI Brief_10월호_산업동향_1002_F.pdf"

# psycopg를 사용하여 vector extension 설치
conn = None

try:
    conn = psycopg.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")  
    conn.commit()
    print("✅ vector extension을 성공적으로 확인/생성했습니다.")
    cursor.close()
except Exception as e:
    print(f"❌ Extension 생성 중 에러 발생: {e}")
finally:
    if conn is not None:
        conn.close()

# PDF 로드 및 텍스트 분할
loader = PyPDFLoader(FILE_PATH)
pages = loader.load()
print(f"PDF 파일을 총 {len(pages)} 페이지로 불러왔습니다.")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents_to_add = text_splitter.split_documents(pages)
print(f"문서를 총 {len(documents_to_add)}개의 청크로 분할했습니다.")

# 랭체인 pgvector 준비
CONNECTION_STRING = PGVector.connection_string_from_db_params(driver="psycopg", **DB_CONFIG) 
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

db = PGVector.from_documents(
    documents=documents,
    embedding=embeddings_model,
    collection_name="ensemble_example",
    connection=CONNECTION_STRING,   
    pre_delete_collection=True
)

print(f"✅ {COLLECTION_NAME} 컬렉션에 문서를 추가 완료했습니다.")

# 유사도 검색
retrieved_docs = db.similarity_search("생성형 AI의 기술 동향에 대해 알려줘", k=3)
print("\n--- 검색 결과 ---")

if retrieved_docs:
    for idx, doc in enumerate(retrieved_docs, 1):
        print(f"\n[결과 {idx}]")
        print(f"내용: {doc.page_content[:200]}...")  
        print(f"출처: {doc.metadata.get('source', '알 수 없음')}")
        print(f"페이지: {doc.metadata.get('page', '알 수 없음')}")
        print("-" * 80)

❌ Extension 생성 중 에러 발생: invalid connection option "database"

PDF 파일을 총 29 페이지로 불러왔습니다.
문서를 총 70개의 청크로 분할했습니다.


Collection not found


✅ vector-db-test 컬렉션에 문서를 추가 완료했습니다.

--- 검색 결과 ---

[결과 1]
내용: 정책･법제기업･산업기술･연구인력･교육
19
MBZUAI와 G42, 추론 AI 모델 ‘K2 Think’ 오픈소스 공개nUAE의 모하메드 빈 자이드 AI 대학(MBZUAI)과 국영 AI 기업 G42가 혁신적인 사후 학습 기법과 테스트-타임 스케일링을 적용한 매개변수 320억 개의 추론 AI 모델 ‘K2 Think’를 공개nK2 Think는 수학 벤치마크에서 딥...
출처: data/SPRi AI Brief_10월호_산업동향_1002_F.pdf
페이지: 20
--------------------------------------------------------------------------------

[결과 2]
내용: 2025년10월호인공지능 산업의 최신 동향...
출처: data/SPRi AI Brief_10월호_산업동향_1002_F.pdf
페이지: 0
--------------------------------------------------------------------------------

[결과 3]
내용: £과학기술, 산업, 소비, 민생, 거버넌스 역량, 국제협력의 6대 영역에서 AI 플러스 추진n중국 국무원이 AI와 경제·사회의 심층 융합을 목표로 ‘AI 플러스(AI+) 행동 심화 추진에 관한 의견’을 발표∙2027년까지 폭넓고 심도 있는 AI 융합의 실현 및 차세대 지능형 단말과 지능형 에이전트 등의 보급률 70% 돌파, 2030년까지는 차세대 지능형 단...
출처: data/SPRi AI Brief_10월호_산업동향_1002_F.pdf
페이지: 3
--------------------------------------------------------------------------------


In [ ]:
import psycopg
from dotenv import load_dotenv
import os
from langchain_openai import OpenAIEmbeddings
from langchain_postgres import PGVector
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

load_dotenv()

DB_CONFIG = {
    'host': os.getenv("DB_HOST"),
    'port': 5432,
    'database': os.getenv('DB_NAME'),  
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASS'),
}

COLLECTION_NAME = "vector-db-test"
FILE_PATH = "data/SPRi AI Brief_10월호_산업동향_1002_F.pdf"

# psycopg를 사용하여 vector extension 설치
conn = None

try:
    conn = psycopg.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    conn.commit()
    print("✅ vector extension을 성공적으로 확인/생성했습니다.")
    cursor.close()
except Exception as e:
    print(f"❌ Extension 생성 중 에러 발생: {e}")
finally:
    if conn is not None:
        conn.close()

# PDF 로드 및 텍스트 분할
loader = PyPDFLoader(FILE_PATH)
pages = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = text_splitter.split_documents(pages)

# Vector store 생성
CONNECTION_STRING = PGVector.connection_string_from_db_params(driver="psycopg", **DB_CONFIG)
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = PGVector.from_documents(
    documents=documents,
    embedding=embeddings_model,
    collection_name="ensemble_example",
    connection=CONNECTION_STRING,   
    pre_delete_collection=True
)

# Vector retriever 생성
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# BM25 retriever 생성
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 3

print("=" * 80)
print("Ensemble Retriever (Hybrid Search)")
print("- Sparse(BM25) + Dense(Vector) 결합")
print("- 키워드와 의미 모두 고려")
print("=" * 80)

# Ensemble Retriever 생성
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.5, 0.5]
)

# 검색 실행 (invoke 오타 수정)
results = ensemble_retriever.invoke("생성형 AI 기술 동향")

# 결과 출력
if results:
    for idx, doc in enumerate(results, 1):
        print(f"\n[결과 {idx}]")
        print(f"내용: {doc.page_content[:200]}...")
        print(f"출처: {doc.metadata.get('source', '알 수 없음')}")
        print(f"페이지: {doc.metadata.get('page', '알 수 없음')}")
        print("-" * 80)

❌ Extension 생성 중 에러 발생: invalid connection option "database"

Ensemble Retriever (Hybrid Search)
- Sparse(BM25) + Dense(Vector) 결합
- 키워드와 의미 모두 고려

[결과 1]
내용: 2025년10월호인공지능 산업의 최신 동향...
출처: data/SPRi AI Brief_10월호_산업동향_1002_F.pdf
페이지: 0
--------------------------------------------------------------------------------

[결과 2]
내용: 보안 취약점을 줄이기 위한 안전 규정 수립과 이행, 공급망 보안 강화, AI 모델 안전 가드레일 구축 등을 진행∙(파생 위험 대책) 친환경 AI 모델 및 컴퓨팅 기술, 기술 표준 개발을 모색하고 생명과 건강에 영향을 미치는 핵심 분야의 AI 시스템에 대한 비상 통제 대책을 마련 n(종합 거버넌스 조치) 기술적 대책 마련과 함께 정부, 사회단체, 사용자 등 ...
출처: data/SPRi AI Brief_10월호_산업동향_1002_F.pdf
페이지: 7
--------------------------------------------------------------------------------

[결과 3]
내용: 정책･법제기업･산업기술･연구인력･교육
19
MBZUAI와 G42, 추론 AI 모델 ‘K2 Think’ 오픈소스 공개nUAE의 모하메드 빈 자이드 AI 대학(MBZUAI)과 국영 AI 기업 G42가 혁신적인 사후 학습 기법과 테스트-타임 스케일링을 적용한 매개변수 320억 개의 추론 AI 모델 ‘K2 Think’를 공개nK2 Think는 수학 벤치마크에서 딥...
출처: data/SPRi AI Brief_10월호_산업동향_1002_F.pdf
페이지: 20
------------------------------------------------------------------

In [3]:
import psycopg
from dotenv import load_dotenv
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_postgres import PGVector
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough  
from langchain_core.output_parsers import StrOutputParser 

load_dotenv()

print("\n[1단계] 문서로드 (Document Loader)")
print("-" * 80)

FILE_PATH = "data/SPRi AI Brief_10월호_산업동향_1002_F.pdf"
loader = PyPDFLoader(FILE_PATH)

documents = loader.load()
print(f"총 {len(documents)} 페이지 로드 완료")

print("\n[2단계] 문서 분할 (Text Splitter)")
print("-" * 80)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len
)

splits = text_splitter.split_documents(documents)
print(f"총 {len(splits)}개의 청크로 분할 완료")
print(f"첫 번째 청크: {splits[0].page_content[:150]}...")

print("\n[3단계] 문서 임베딩 (Document Embedding)")
print("-" * 80)

embeddings_model = OpenAIEmbeddings(model='text-embedding-3-small')

sample_text = "생성형 AI 기술동향"
sample_vector = embeddings_model.embed_query(sample_text)

print("임베딩 모델 준비 완료")
print(f"벡터 차원: {len(sample_vector)}차원")
print(f"샘플 벡터(처음 5개): {sample_vector[:5]}")

print("\n[4단계] Postgres 벡터 확장 확인")
print("-" * 80)

DB_CONFIG = {
    'host': os.getenv("DB_HOST"),
    'port': 5432,
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASS'),
}

conn = None

try:
    conn = psycopg.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    conn.commit()
    print("✅ vector extension을 성공적으로 확인/생성했습니다.")
    cursor.close()
except Exception as e:
    print(f"❌ Extension 생성 중 에러 발생: {e}")
finally:
    if conn is not None:
        conn.close()

print("\n[5단계] PGVector 저장 (Vector Store)")
print("-" * 80)

CONNECTION_STRING = (
    f"postgresql+psycopg://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
    f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"
)

vectorstore = PGVector.from_documents(
    documents=splits,
    embedding=embeddings_model,
    collection_name="rag_example",
    connection=CONNECTION_STRING,
    pre_delete_collection=True
)

print(f"✅ 벡터스토어에 {len(splits)}개 청크 저장 완료")
print("컬렉션 이름: rag_example")

print("\n[6단계] 검색기(Retriever)")
print("-" * 80)

retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={"k": 3}
)

test_query = "생성형 AI의 최신 기술 동향은?"
retrieved_docs = retriever.invoke(test_query)

print("검색기 생성 완료")
print(f"테스트 쿼리: {test_query}")
print(f"검색된 문서 수: {len(retrieved_docs)}")
print(f"첫 번째 결과: {retrieved_docs[0].page_content[:100]}...")

print("\n[7단계] 프롬프트(Prompt)")
print("-" * 80)

prompt_template = ChatPromptTemplate.from_messages([
    (
        "system",
        """당신은 AI 기술 전문가입니다.
아래 제공된 문맥(Context)을 반드시 참고하여 질문에 답변하세요.

중요:
1. 문맥에 관련 내용이 있으면 그것을 바탕으로 답변하세요.
2. 문맥에 없는 내용만 "제공된 문서에서 해당 정보를 찾을 수 없습니다."라고 답변하세요.
3. 답변 시 문맥의 어느 부분을 참고했는지 명시하세요.

문맥:
{context}
"""
    ),
    ("human", "{question}")
])

print("프롬프트 템플릿 생성 완료")
print("시스템 역할: AI 기술 전문가")
print("프롬프트 구조: 문맥(context) + 질문(question)")

print("\n[8단계] LLM(Large Language Model)")
print("-" * 80)

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.5)

print("LLM 모델 준비 완료")
print("모델: gpt-4o-mini")

print("\n[9단계] RAG 체인 구성")
print("-" * 80)


def format_docs(docs):
    formatted = []
    for idx, doc in enumerate(docs, 1):
        formatted.append(f"[문서 {idx}]\n{doc.page_content}")
    return "\n\n---\n\n".join(formatted)

# ✅ 체인 구성
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt_template
    | llm
    | StrOutputParser()
)

print("RAG 체인 구성 완료 ✅")


# RAG 실행 테스트
print("RAG 시스템 실행 테스트")
print("=" * 80)

# 질문 목록
question_list = [
    "중국 국무원이 발표한 'AI플러스' 정책의 3단계 중장기 목표는 무엇이며, 6대 핵심 영역은 어디인가요?",
    "구글이 공개한 이미지 편집 모델 '제미나이 2.5 플래시 이미지'의 가장 큰 특징은 무엇인가요?",
    "스탠포드 대학의 연구결과, 생성 AI의 확산이 경력 초기 근로자의 고용에 어떤 영향을 미치고 있나요?"
]

for idx, question in enumerate(question_list, 1):
    print(f"\n[질문 {idx}] {question}")
    print("-" * 80)
    
    # RAG chain 실행
    answer = rag_chain.invoke(question)
    
    print(f"답변:\n{answer}")
    print("-" * 80)



[1단계] 문서로드 (Document Loader)
--------------------------------------------------------------------------------
총 29 페이지 로드 완료

[2단계] 문서 분할 (Text Splitter)
--------------------------------------------------------------------------------
총 70개의 청크로 분할 완료
첫 번째 청크: 2025년10월호인공지능 산업의 최신 동향...

[3단계] 문서 임베딩 (Document Embedding)
--------------------------------------------------------------------------------
임베딩 모델 준비 완료
벡터 차원: 1536차원
샘플 벡터(처음 5개): [0.04349127784371376, 0.0312628448009491, 0.02276095747947693, 0.0059245433658361435, 0.0064712525345385075]

[4단계] Postgres 벡터 확장 확인
--------------------------------------------------------------------------------
✅ vector extension을 성공적으로 확인/생성했습니다.

[5단계] PGVector 저장 (Vector Store)
--------------------------------------------------------------------------------
✅ 벡터스토어에 70개 청크 저장 완료
컬렉션 이름: rag_example

[6단계] 검색기(Retriever)
--------------------------------------------------------------------------------
검색기 생성 완료
테스트 쿼리: 생성형 AI의 최신 기술 동향은?
검색된 문